In [73]:
import time
import torch
from torch import nn, optim
from torchvision import datasets, transforms

batch_size = 5000

train_transforms = transforms.ToTensor()
test_transforms = transforms.ToTensor()

train_data = datasets.MNIST(root='data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [74]:
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1= nn.Linear(784, 512)
        self.fc2= nn.Linear(512, 512)
        self.fc3= nn.Linear(512, 10)
        
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        
        return x

model = Network()
model

Network(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [75]:
device = 'mps' if torch.has_mps else 'cpu'
certerion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
print(device)

mps


In [71]:
device = 'cpu'
certerion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
print(device)

cpu


In [76]:
epoch = 30
step = 0
print_every = 10
training_loss = 0

model.to(device)
model.train()
start = time.time()

print(device)

for e in range(epoch):
    for images, labels in trainloader:
        step += 1 
        
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad() 
        output = model(images)
        loss = certerion(output, labels)
        loss.backward()
        optimizer.step()
        
        training_loss += loss.item()*images.size(0)
        
        if (step % print_every) == 0:
            accuracy = 0
            model.eval()
            with torch.no_grad():
                testing_loss = 0
                for images, labels in testloader:
                    images, labels = images.to(device), labels.to(device)
                    
                    output = model(images)
                    loss = certerion(output, labels)
                    ps = F.softmax(output, dim=1)
                    top_score, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(top_class.shape)
                    accuracy += torch.mean(equals.float())
                    
                    testing_loss += loss.item()*images.size(0)
                    
                    
                    
            print(f'epoch: {e+1} / {epoch}..  '
                  f'training_loss: {training_loss / len(trainloader.dataset):.3f}..  '
                  f'testing_loss: {testing_loss / len(testloader.dataset):.3f}..  '
                  f'accuracy: {accuracy /len(testloader) * 100:.3f}%  '
                  f'time needed: {(time.time() - start):.3f}')
            
            start = time.time()
            training_loss = 0
            model.train()
            
            

mps
epoch: 1 / 30..  training_loss: 0.984..  testing_loss: 0.482..  accuracy: 85.970%  time needed: 1.617
epoch: 2 / 30..  training_loss: 0.355..  testing_loss: 0.306..  accuracy: 91.090%  time needed: 1.543
epoch: 3 / 30..  training_loss: 0.247..  testing_loss: 0.238..  accuracy: 92.700%  time needed: 1.533
epoch: 4 / 30..  training_loss: 0.188..  testing_loss: 0.184..  accuracy: 94.350%  time needed: 1.529
epoch: 5 / 30..  training_loss: 0.152..  testing_loss: 0.151..  accuracy: 95.270%  time needed: 1.565
epoch: 5 / 30..  training_loss: 0.128..  testing_loss: 0.129..  accuracy: 96.040%  time needed: 1.501
epoch: 6 / 30..  training_loss: 0.110..  testing_loss: 0.117..  accuracy: 96.430%  time needed: 1.516
epoch: 7 / 30..  training_loss: 0.091..  testing_loss: 0.103..  accuracy: 96.740%  time needed: 1.530
epoch: 8 / 30..  training_loss: 0.080..  testing_loss: 0.093..  accuracy: 97.010%  time needed: 1.517
epoch: 9 / 30..  training_loss: 0.071..  testing_loss: 0.086..  accuracy: 97.2

In [8]:
a = torch.tensor([[1,2],
                  [3,4],
                  [5,6]])
b = torch.tensor([[1,2],
                  [3,3],
                  [5,7]])

c = a == b
print(c)

tensor([[ True,  True],
        [ True, False],
        [ True, False]])


In [2]:
import time
import torch
from torchvision import datasets, transforms, models
from torch import nn, optim
from collections import OrderedDict
import opendatasets as od

od.download("https://www.kaggle.com/competitions/dogs-vs-cats/data")

train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])


data_dir = 'Cat_Dog_data'

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
trainlader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: tensun
Your Kaggle Key: ········


100%|█████████████████████████████████████████| 812M/812M [28:05<00:00, 505kB/s]



Extracting archive ./dogs-vs-cats/dogs-vs-cats.zip to ./dogs-vs-cats


FileNotFoundError: [Errno 2] No such file or directory: 'Cat_Dog_data/train'

In [6]:

model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 500)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(500, 2)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.classifier = classifier

device = "mps"

model.to(device)

certerion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

epoch = 1
print_every = 5
step = 0

for ii, (inputs, labels) in enumerate():

    inputs, labels = inpute.to(device), labels.to(device)

    start = time.time()

    optimizer.zero_grad()

    output = model(inputs)
    loss = certerion(output, labels)
    loss.backward()
    optimizer.step()

    if ii == 3:
        break

    print(f'Device: {device}; Time per batch: {(time.time() - start)/3:.3f} seconds')



In [8]:
torch.has_mps

True

In [5]:
print(
    f'hello.. '
    f'eric'
)

hello.. eric


In [ ]:
    this is the first line of code inlind with text


    this is a line of code inline with text
LOL what was that singing

In [ ]:
````
print('hello')
````

markdowwwwn


````
code block
````

In [ ]:
import numpy as np
a = np.array([1,0,1])[:,None]
c = 3
print(a)

In [ ]:
from collections import Counter

a = "The Dog jumps over the roof".lower()
b = "The cat jumps on the roof".lower()

vocab = Counter()
for word in a.split(' '):
    vocab[word] += 1
    
print(vocab)
print(len(vocab))

for i,word in enumerate(vocab):
    print(i,vocab[word])

In [ ]:
vocab = set()
vocab.add("ehllo")
vocab.add('ehllo')
vocab.add("Hello")
vocab.add("hello")
print(vocab)
for i,word in enumerate(vocab):
    print(i, word)
vocab_list = list(vocab)
print(vocab_list)
for i,word in enumerate(vocab_list):
    print(i, word)

In [ ]:
init_weights = np.zeros([4,1])
print(init_weights)


In [ ]:
class Student:
    def __init__(self,age,name,update_age):
        self.age = age
        self.name = name
        self.update_age(update_age)
    def update_age(self, age):
        self.age = age
        print('Done! Your age is: ', self.age)
student1 = Student(18, 'eric', 19)

In [ ]:
assert(2==1)

In [ ]:
import numpy as np

a = np.array([[1,2,3]])
b = np.array([[4],
              [5],
              [6]])
c = 10
print(b.dot(a))

In [ ]:
a = np.random.rand(2,2)
b = np.zeros((2,2))
print(a.dot(10))
print(b)

In [ ]:
a = "Cute dog"
if ("dog" in a.split(' ')):
    print("something")

In [ ]:
zeros = np.zeros((1,10))
print(zeros)
print(zeros.size)
a = np.array([1,2,3,4])
print(a)
print(a.size)

zeros[8] += 1

In [ ]:
a = np.array([1,2,3,4,5,6,7,8,9])
print(a[:-2])
print(a[-2:])
print('POSITIVE' == "POSITIVE")

In [ ]:
a = [1]
a[9] = 10
print(a)

In [ ]:
a = "german shepherd is the cutest"
for i,word in enumerate(a.split(' ')):
    print(i,word)

In [ ]:
a = np.random.rand(3,3)
print(a)
print(a[1][:])
print(a[1])

In [ ]:
indices = np.zeros((1, 2))
print(indices)
indices[0][0] = 4
indices[0][1] = 10
print(indices)

for index in indices[0]:
    print(index)

In [ ]:
a = "I like dogs and cats"
for i,word in enumerate(a.split(' ')):
    print(i, word)
    print(type(i))
    
    

In [ ]:
a = set([1,2,3])
print(a)
b = list(a)
print(b)
print(type(b[1]))

In [ ]:
from collections import Counter
a = Counter()
review = "German shepherd is the cutest dog dog :D"
for word in review.split(' '):
    a[word] += 1
print(a)

word_count2 = Counter()

for word,cnt in list(a.most_common()):
    print(word)
    print(cnt)


In [ ]:
a = np.array([[1,2,3]])
b = np.array(2)
print(a.dot(b))
print(b.dot(a))

In [ ]:
if 1<2 and 2>1:
    print("correct")

In [ ]:
a = "dog cat"
if "dog" in a and 
    print(a)

In [ ]:
for i in range 5:
    print(i)

In [ ]:
x = np.array([1,2,3,4,5,6])
y = [1,2,3,4,5,6]
hidden_error_term = [[1,2],
                     [3,4]]
#print(hidden_error_term *x[:, None])

print(np.dot([2], x))
print(x*[2])

In [ ]:
x = np.array([1,2])
y = np.array([[1],
     [2]])
c = np.array([2,3])

output = np.dot(x, y)
print(output)
print(output - 2)

hidden_error = np.dot(y, output)
hidden_error_term = hidden_error *c

print(hidden_error)
print(1- hidden_error_term)


In [ ]:
x = np.array([1])
y = np.array([3,4])

print(np.dot(x, [y]))
print(y.T)




In [ ]:
import torch
a = torch.randn((2,3))
print(a)

In [ ]:
from datasets import load_dataset  # pip install datasets

# load the first 1K rows of the TREC dataset
trec = load_dataset('trec', split='train[:1000]')
trec

In [ ]:
from transformers import AutoTokenizer, AutoModel  # pip install transformers

# initialize BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# take the first 64 rows of the trec data
text = trec['text'][:64]
# tokenize text using the BERT tokenizer
tokens = tokenizer(
    text, max_length=512,
    truncation=True, padding=True,
    return_tensors='pt'
)

In [ ]:
torch.has_mps

In [ ]:
device = torch.device('mps')
model.to(device)
tokens.to(device)
device

In [ ]:
%%timeit
model(**tokens)